In [1]:
import tweepy
from config import BEARER_TOKEN
import time
import pandas as pd

In [2]:
client = tweepy.Client(BEARER_TOKEN, wait_on_rate_limit=True)

In [3]:
btc_query = '(#btc OR #bitcoin OR bitcoin OR $btc ) -is:retweet -is:reply -has:links lang:en'

eth_query = '(#eth OR #ethereum OR eth or ethereum OR $eth)-is:retweet -is:reply -has:links lang:en'

In [4]:
hoax_tweets = []
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = btc_query,
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2022-03-21T00:00:00Z',
                                 end_time = '2022-04-21T00:00:00Z',
                              max_results=500):
    time.sleep(1)
    hoax_tweets.append(response)

Rate limit exceeded. Sleeping for 270 seconds.
Rate limit exceeded. Sleeping for 269 seconds.
Rate limit exceeded. Sleeping for 261 seconds.
Rate limit exceeded. Sleeping for 265 seconds.


In [5]:
result = []
user_dict = {}
# Loop through each response object
for response in hoax_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                             }
    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet.author_id, 
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'author_description': author_info['description'],
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                      })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638728 entries, 0 to 638727
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   author_id           638728 non-null  int64              
 1   username            638728 non-null  object             
 2   author_followers    638728 non-null  int64              
 3   author_tweets       638728 non-null  int64              
 4   author_description  638728 non-null  object             
 5   author_location     326583 non-null  object             
 6   text                638728 non-null  object             
 7   created_at          638728 non-null  datetime64[ns, UTC]
 8   retweets            638728 non-null  int64              
 9   replies             638728 non-null  int64              
 10  likes               638728 non-null  int64              
 11  quote_count         638728 non-null  int64              
dtypes: datetime64[ns

In [7]:
df.to_csv("btc_data_2022_val.csv")

In [8]:
df

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count
0,1460249327394164739,devvaaaaaaa,10,664,"ᖴᖇᝪᗰ Ꭲᕼᗴ ᗴᗩᔑᎢ ᑕᝪᗩᔑᎢ, しᏆᐯᗴ ᗷᎩ ᗴ᙭ᗩᗰᑭしᗴ, ᑕᝪᑎᔑᗴᖇᐯᗩ...",None,Want to wake up in the morning without financi...,2022-04-20 23:59:55+00:00,12,1,18,0
1,1501922448324927500,LivePriceCrypto,1272,67379,I Tweet the latest #Bitcoin & #Ethereum price ...,None,#Bitcoin Last Price $41368 #BTC 🚀\nDaily Indic...,2022-04-20 23:59:52+00:00,0,0,0,0
2,1502555711460495362,theashleepage11,330,762,Deep Value Investor . . . 📄📑📉👩‍💻 NFTs Advocate...,Ocean city,Give a man #bitcoin and he trades it. Teach a...,2022-04-20 23:59:52+00:00,0,0,3,0
3,1502555711460495362,theashleepage11,330,762,Deep Value Investor . . . 📄📑📉👩‍💻 NFTs Advocate...,Ocean city,If you think there is a point where bitcoin is...,2022-04-20 23:59:48+00:00,0,1,4,0
4,771448125651247104,SkylerColbert,1872,1039,Community Builder | Accumulating & Busy @ @Luv...,"Costa Mesa, CA",A lot of people are unaware of how early we al...,2022-04-20 23:59:46+00:00,4,2,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...
638723,51904755,StoreFiAi,764,52411,#Cryptocurrency Live Prices #bitcoin #doge #et...,None,Top 10 #Cryptocurrency Live Prices: \n(08:00 P...,2022-03-21 00:00:00+00:00,0,0,0,0
638724,1464382392223866884,AlertingCryptos,2615,48841,I'm alerting you of any relevant info about cr...,None,Most mentionned cryptos on reddit today so far...,2022-03-21 00:00:00+00:00,0,0,0,0
638725,1117701695688060929,BinanceRekts,1067,9822,We are watching bitmex and bybit for anything ...,None,🔉 ⚠️ $BTC 4 Hours Update ⚠️ 🔉\nPrice @ $41268 ...,2022-03-21 00:00:00+00:00,0,0,0,0
638726,1117701695688060929,BinanceRekts,1067,9822,We are watching bitmex and bybit for anything ...,None,🔉 ⚠️ $BTC Weekly Update ⚠️ 🔉\nPrice @ $41268 (...,2022-03-21 00:00:00+00:00,0,0,0,0


In [9]:
# import numpy as np
# df.to_csv("btc_bitcoin.csv")